In [1]:
#!/usr/bin/python

import numpy as np
import math
from astropy.io import fits as fits

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

import pandas as pd
from pandas import DataFrame, Series

from astropy.wcs import WCS as WCS

import import_ipynb
import Pipe3D_Asier_Test_II as Pipe3D

from mpdaf.obj import hms2deg
from mpdaf.obj import dms2deg

from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

from scipy.stats import ks_2samp
from scipy import stats

importing Jupyter notebook from Pipe3D_Asier_Test_II.ipynb


In [2]:
alfa = 1.0
delta = 0.0
width = [0.1,25,50,75,100,150,200,500]

In [3]:
for sigma in width:
    print(sigma)

    #Crear arrays con los datos de todas las galaxias.
    Numb_Lines = len(open('Names_Amusing_true.txt').readlines())

    All_SN_type = np.zeros(Numb_Lines)
    All_Prob_II = np.zeros(Numb_Lines)
    All_Cumul_II = np.zeros(Numb_Lines)

    names = open('Names_Amusing_true.txt')

    Galaxy = -1
    nIa = 0
    nII = 0
    for lines in names:
        line= lines[:-1]
        print(line)
        Galaxy = Galaxy+1

        df = pd.read_csv('Amusing_Table_true.csv')  # To find the Galaxy data
        Names_1 = df['SN_name']
        Names_2 = df['def_name']
        R = 0

        for names in Names_1:              # Finding the Galaxy in the DataFrame
            R=R+1
            if line == names:
                Row = R-1
        R = 0
        for names in Names_2:              # Some Galaxies have the name in other column
            R=R+1
            if line == names:
                Row = R-1

        # SN coordinates
        RA_SN = df.at[Row,'RA_SN']
        RA_SN = hms2deg(RA_SN)

        DEC_SN = df.at[Row,'DEC_SN']
        DEC_SN = dms2deg(DEC_SN)

        # Galaxy center coordinates
        RA_GAL = df.at[Row,'RA_GAL']
        RA_GAL = hms2deg(RA_GAL)

        DEC_GAL = df.at[Row,'DEC_GAL']
        DEC_GAL = dms2deg(DEC_GAL)

        # Pixel position
        w = WCS('../Data/Amusing/flux_elines.'+line+'.cube.fits')
        Pix_X, Pix_Y ,z = w.wcs_world2pix(RA_SN, DEC_SN, 0. , 1)
        Pix_X_GAL, Pix_Y_GAL ,z = w.wcs_world2pix(RA_GAL, DEC_GAL, 0. , 1)

        # Integer for measure
        Pix_X = int(Pix_X)
        Pix_Y = int(Pix_Y)
        
        #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
        # Pixel de la SN
        Pix_X = df.at[Row,'SNX'] -1
        Pix_Y = df.at[Row,'SNY'] -1
        # Pixel del centro de la Galaxia
        Pix_X_GAL = df.at[Row,'GALX'] -1
        Pix_Y_GAL = df.at[Row,'GALY'] -1
        #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

        # Supernova type
        if df.at[Row,'SN_type'] == 'Ia':
            SN_TYPE = 1
            nIa = nIa + 1
        elif df.at[Row,'SN_type'] == 'II':
            SN_TYPE = 2
            nII = nII + 1
        else:
            SN_TYPE = 0

    #~~~~~~~~~~~~ SN Rates ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        # Call to Pipe3D code
        line = Pipe3D.Pipe3D_galaxy(line)

        # Obtain the maps of the galaxy
        line.painting_galaxy_maps(False, alfa, delta, sigma)

        # Gaussian function
        stddev = 7
        gauss = Gaussian2DKernel(stddev)

        # Gaussian convolve
        line.SNII_rate_map = convolve(line.SNII_rate_map, gauss)

        # Eliminate negative values
        line.SNII_rate_map[line.SNII_rate_map<0.0] = 0.0

    # ------------- SN Ia ------------------------------------------------------------------------------------

        # An Array whit all the value of the SN Ia rate map
        line.Prob_List_II = np.ndarray.flatten(line.SNII_rate_map)
        # The array orther in value
        line.Order_Prob_List_II = np.sort(line.Prob_List_II)

        # Normalize
        line.Norm_SNII = 0
        for i in range(len(line.Order_Prob_List_II)):
            line.Norm_SNII = line.Norm_SNII + line.Order_Prob_List_II[i]

        line.Order_Prob_List_II = line.Order_Prob_List_II / line.Norm_SNII
        line.SNII_rate_map = line.SNII_rate_map / line.Norm_SNII

        # Calculate P
        line.cumulative_P_values_II = np.cumsum(line.Order_Prob_List_II)

        # Calculate P map
        line.cumulative_SNII_rate_map = np.interp(line.SNII_rate_map, line.Order_Prob_List_II, line.cumulative_P_values_II)

        # Measure Probability
        line.Prob_Val_SNII = 0
        line.Cumulative_Prob_SNII = 0
        square = np.arange(3)-1

        for j in square:
            for i in square:
                # The probability of SNIa (p)
                line.Prob_Val_SNII = line.Prob_Val_SNII + line.SNII_rate_map[Pix_Y+j,Pix_X+i]
                # The cumulative probability of SNIa (P)
                line.Cumulative_Prob_SNII = line.Cumulative_Prob_SNII + line.cumulative_SNII_rate_map[Pix_Y+j,Pix_X+i]

        line.Prob_Val_SNII = line.Prob_Val_SNII / 9
        line.Cumulative_Prob_SNII = line.Cumulative_Prob_SNII/9

    # ------------- Save Data ------------------------------------------------------------------------------------

        # SN type
        All_SN_type[Galaxy] = SN_TYPE
        # SN Ia and cumulative SN Ia Probability
        All_Prob_II[Galaxy] = line.Prob_Val_SNII
        All_Cumul_II[Galaxy] = line.Cumulative_Prob_SNII

        data_II = np.column_stack((All_SN_type,All_Prob_II,All_Cumul_II))
        np.savetxt('../Results_CC_'+str(sigma)+'.coordenadas.txt',data_II)

0.1
ASASSN13an


/home/asier/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:125: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/asier/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:127: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ASASSN13ch
ASASSN13cj
ASASSN13cp
ASASSN13dm
ASASSN14at
ASASSN14ba
ASASSN14bf
ASASSN14co
ASASSN14cu
ASASSN14db
ASASSN14dz
ASASSN14hr
ASASSN14ig
ASASSN14ko
ASASSN14lt
ASASSN14lu
ASASSN14lv
ASASSN15aj
CSP14aaq


a floating-point value was expected. [astropy.wcs.wcs]


SN1970A
SN1985G
SN1990Q
SN1997dn
SN1998bw
SN2001fv
SN2004ef
SN2004gs
SN2005ag
SN2005bg
SN2005hc
SN2005ku
SN2005M
SN2005na
SN2006br
SN2006D
SN2006ej
SN2006et
SN2006hx
SN2006lu
SN2006my
SN2006os
SN2006ot
SN2007al
SN2007ba
SN2007bc
SN2007cg
SN2007hj
SN2007ol
SN2009bb
SN2009I
SN2010F
SN2012gm
SN2012ho
SN2014cd
25
ASASSN13an
ASASSN13ch
ASASSN13cj
ASASSN13cp
ASASSN13dm
ASASSN14at
ASASSN14ba
ASASSN14bf
ASASSN14co
ASASSN14cu
ASASSN14db
ASASSN14dz
ASASSN14hr
ASASSN14ig
ASASSN14ko
ASASSN14lt
ASASSN14lu
ASASSN14lv
ASASSN15aj


KeyboardInterrupt: 